In [ ]:
!pip install pandas numpy matplotlib seaborn scikit-learn statsmodels interpret


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 38.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.1/780.1 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 16.7 MB/s eta 0:00:00
  Created wheel for dash-cytoscape: filename=dash_cytoscape-1.0.2-py3-none-any.whl size=4

This notebook implements a complete data pipeline, forecasting models, and explainability analysis for patient mobility data.

In [ ]:
import json
import pandas as pd

with open("timeseries-data.json") as f:
    ts_data = json.load(f)

with open("categorical-data.json") as f:
    clinical_data = json.load(f)

print(len(ts_data))
print(clinical_data.keys())


80919
dict_keys(['gender', 'isSmoker', 'birthYear', 'birthMonth', 'disease', 'diagnoses', 'events', 'sideEffects', 'therapies', 'molecularAnalysisSelection'])


In [ ]:
ts_df = pd.DataFrame(ts_data)
ts_df["start"] = pd.to_datetime(ts_df["start"])
ts_df["end"] = pd.to_datetime(ts_df["end"])


In [ ]:
ts_df["date"] = ts_df["start"].dt.date

daily_steps = (
    ts_df.groupby("date")["count"]
    .sum()
    .reset_index()
    .rename(columns={"count": "Daily_Step_Count"})
)

daily_steps["date"] = pd.to_datetime(daily_steps["date"])
daily_steps.head()


,date,Daily_Step_Count
0,2021-10-10,19980
1,2021-10-11,11229
2,2021-10-12,15117
3,2021-10-13,13527
4,2021-10-14,17781


In [ ]:
full_dates = pd.date_range(
    start=daily_steps["date"].min(),
    end=daily_steps["date"].max(),
    freq="D"
)

daily_steps = (
    daily_steps
    .set_index("date")
    .reindex(full_dates)
    .fillna(0)
    .rename_axis("date")
    .reset_index()
)


In [ ]:
gender = clinical_data["gender"]
birth_year = clinical_data["birthYear"]
disease = clinical_data["disease"]

daily_steps["age"] = 2025 - birth_year
daily_steps["gender_M"] = 1 if gender == "M" else 0
daily_steps["gender_F"] = 1 if gender == "F" else 0


In [ ]:
therapies = clinical_data.get("therapies", [])

daily_steps["date"] = pd.to_datetime(daily_steps["date"]).dt.tz_localize(None)

for i, therapy in enumerate(therapies):
    col = f"is_on_therapy_{i+1}"
    daily_steps[col] = 0

    start = pd.to_datetime(therapy["startDate"]).tz_localize(None)

    # ✅ FIX: handle None endDate
    if therapy["endDate"] is None:
        end = daily_steps["date"].max()
    else:
        end = pd.to_datetime(therapy["endDate"]).tz_localize(None)

    daily_steps.loc[
        (daily_steps["date"] >= start) &
        (daily_steps["date"] <= end),
        col
    ] = 1




In [ ]:
side_effects = clinical_data.get("sideEffects", [])

daily_steps["active_side_effect_count"] = 0
daily_steps["max_side_effect_intensity"] = 0

for se in side_effects:
    start = pd.to_datetime(se["startDate"], utc=True).tz_localize(None)

    # handle None endDate
    if se["endDate"] is None:
        end = daily_steps["date"].max()
    else:
        end = pd.to_datetime(se["endDate"], utc=True).tz_localize(None)

    intensity = se.get("intensity", 0)

    mask = (daily_steps["date"] >= start) & (daily_steps["date"] <= end)

    daily_steps.loc[mask, "active_side_effect_count"] += 1
    daily_steps.loc[mask, "max_side_effect_intensity"] = (
        daily_steps.loc[mask, "max_side_effect_intensity"]
        .combine(pd.Series(intensity, index=daily_steps.loc[mask].index), max)
    )


In [ ]:
daily_steps["day_of_week"] = daily_steps["date"].dt.dayofweek
daily_steps["week_of_year"] = daily_steps["date"].dt.isocalendar().week.astype(int)

daily_steps["steps_t-1"] = daily_steps["Daily_Step_Count"].shift(1)
daily_steps["steps_t-7"] = daily_steps["Daily_Step_Count"].shift(7)
daily_steps["steps_t-30"] = daily_steps["Daily_Step_Count"].shift(30)

daily_steps = daily_steps.dropna()


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

train = daily_steps.iloc[:-365]
test = daily_steps.iloc[-365:]

model = SARIMAX(
    train["Daily_Step_Count"],
    order=(1,1,1),
    seasonal_order=(1,1,1,7)
)

result = model.fit(disp=False)
forecast = result.forecast(steps=365)


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

rmse = np.sqrt(mean_squared_error(test["Daily_Step_Count"], forecast))
mae = mean_absolute_error(test["Daily_Step_Count"], forecast)

rmse, mae


(np.float64(11011.935702481822), 7575.646458316153)

In [ ]:
from interpret.glassbox import ExplainableBoostingRegressor

features = daily_steps.drop(columns=["date", "Daily_Step_Count"])
target = daily_steps["Daily_Step_Count"]

X_train = features.iloc[:-365]
X_test = features.iloc[-365:]
y_train = target.iloc[:-365]
y_test = target.iloc[-365:]

ebm = ExplainableBoostingRegressor(random_state=42)
ebm.fit(X_train, y_train)

preds = ebm.predict(X_test)


In [ ]:
rmse_ebm = np.sqrt(mean_squared_error(y_test, preds))
mae_ebm = mean_absolute_error(y_test, preds)

rmse_ebm, mae_ebm


(np.float64(7828.485636844886), 5600.727763052091)

In [ ]:
from interpret import show

show(ebm.explain_global())


In [ ]:
future_dates = pd.date_range(
    daily_steps["date"].max() + pd.Timedelta(days=1),
    periods=365
)

forecast_df = pd.DataFrame({
    "Date": future_dates,
    "Predicted_Steps": preds[:365],
    "Trend_Component": forecast.values[:365],
    "Exogenous_Impact": preds[:365] - forecast.values[:365]
})

forecast_df.head()


,Date,Predicted_Steps,Trend_Component,Exogenous_Impact
0,2025-10-22,13305.365502,11119.428283,2185.937219
1,2025-10-23,24171.488832,8701.861315,15469.627516
2,2025-10-24,11346.239378,8464.099222,2882.140155
3,2025-10-25,8884.053209,7283.220542,1600.832667
4,2025-10-26,10016.092976,6265.118822,3750.974154


In [ ]:
def upload_forecast_to_s3(df, bucket_name, file_name):
    import boto3
    import io

    # s3 = boto3.client("s3")
    # csv_buffer = io.StringIO()
    # df.to_csv(csv_buffer, index=False)
    # s3.put_object(Bucket=bucket_name, Key=file_name, Body=csv_buffer.getvalue())

    pass
